In [7]:
import h5py
data_name = 'PBS45p_q10_5000_SVD_60_80dB_ensemble1'
data_name = 'PBS45p_q10'
data_path = '\\\\ls710dc3b\\share\\OmuratoWang\\'
with h5py.File(data_path+data_name+'.mat', 'r') as file:
    # datas = file['Acq']['Data'][:]
    datas = file['ImgBuf3'][:]
    print(type(datas))

<class 'numpy.ndarray'>


In [9]:
print(datas.shape)
print(datas)

(5001, 192, 1216)
[[[ 0.0000e+00  5.0000e+00  1.5000e+01 ...  1.1000e+02  6.3500e+02
   -2.4434e+04]
  [ 0.0000e+00  5.0000e+00  4.0000e+00 ...  1.5800e+02 -3.5200e+02
    9.6000e+02]
  [ 1.0752e+04  5.0000e+00  6.0000e+00 ... -5.7600e+02  2.2300e+02
   -1.5732e+04]
  ...
  [ 0.0000e+00 -3.0000e+00 -8.0000e+00 ... -2.9700e+02 -1.6800e+02
   -2.3718e+04]
  [ 0.0000e+00  3.0000e+00  4.0000e+00 ...  3.8300e+02  5.2000e+01
   -1.7826e+04]
  [ 0.0000e+00  0.0000e+00  2.0000e+00 ...  1.2200e+02  7.1000e+02
   -1.4315e+04]]

 [[ 0.0000e+00  4.0000e+00  1.2000e+01 ...  1.1600e+02  6.4800e+02
   -1.9838e+04]
  [ 0.0000e+00  7.0000e+00  7.0000e+00 ...  1.3200e+02 -4.0200e+02
   -3.1614e+04]
  [ 1.1008e+04  6.0000e+00  1.0000e+01 ... -6.1200e+02  2.3500e+02
   -1.1030e+04]
  ...
  [ 0.0000e+00 -6.0000e+00 -1.3000e+01 ... -2.8900e+02 -1.7400e+02
    1.5266e+04]
  [ 0.0000e+00  2.0000e+00  3.0000e+00 ...  3.9100e+02  7.2000e+01
   -1.5172e+04]
  [ 0.0000e+00 -2.0000e+00  2.0000e+00 ...  1.2700e+02 

In [ ]:
print(type(datas))

In [ ]:
import numpy as np
# 将 NumPy 数组保存到磁盘
np.save('PBS45p_q10_5000_SVD_60_80dB_ensemble1_Acq_Data.npy', datas)

In [10]:
import numpy as np
# 将 NumPy 数组保存到磁盘
np.save(data_name+'.npy', datas)

In [1]:
import numpy as np
# 从磁盘加载 NumPy 数组
data_array = np.load('PBS45p_q10.npy')  
data_array = data_array.astype('float32')
data_array = data_array.transpose(0 ,2, 1)
print(data_array.shape)  # (5001, 192, 1216)

(5001, 1216, 192)


In [2]:
import numpy as np
# 从磁盘加载 NumPy 数组
label_array = np.load('PBS45p_q10_5000_SVD_60_80dB_ensemble1.npy') 
label_array = label_array.astype('float32') 
label_array = label_array.transpose(0, 2, 1)
print(label_array.shape)  # (5000, 192, 1216)

(5000, 1216, 192)


In [3]:
import torch
from torch.utils.data import Dataset, DataLoader  #, TensorDataset

class CustomDataset(Dataset):
    def __init__(self, data_array, label_array, device):
        self.data = torch.from_numpy(data_array).float().unsqueeze(1)#.to(device)
        self.label = torch.from_numpy(label_array).float().unsqueeze(1)#.to(device)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.label[idx]

device = "cuda"
seq_train, seq_val, seq_test = int(5000*0.7), int(5000*0.1), int(5000*0.2)
# data_tensor = torch.from_numpy(data_array)#.half()
# label_tensor = torch.from_numpy(label_array)#.half()
# data_tensor = data_tensor.unsqueeze(1).to(device)
# label_tensor = label_tensor.unsqueeze(1).to(device)
train_dataset = CustomDataset(data_array[:seq_train], label_array[:seq_train], device)
val_dataset = CustomDataset(data_array[seq_train:seq_train+seq_val], label_array[seq_train:seq_train+seq_val], device)
test_dataset = CustomDataset(data_array[seq_train+seq_val:seq_train+seq_val+seq_test], label_array[seq_train+seq_val:seq_train+seq_val+seq_test], device)

batch_size = 4
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [5]:
import torch
from mlp_mixer import MLPMixer_inv

# model = MLPMixer(
#     image_size = (1216, 192),
#     channels = 1,
#     patch_size = 16,
#     dim = 1216,
#     depth = 6,
#     num_classes = 1000
# )

model = MLPMixer_inv(
    image_size = (1216, 192),
    channels = 1,
    patch_size = 16,
    dim = 192,
    depth = 12,
    num_classes = 1000
)

img = torch.randn(1, 1, 1216, 192)
output_tensor = model(img) # (1, 1000)
print(output_tensor.shape)

ModuleNotFoundError: No module named 'einops'

In [4]:
import torch
import torch.nn as nn

class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()

        # Encoder (Downsampling path)
        self.encoder1 = self.conv_block(1, 64)
        self.encoder2 = self.conv_block(64, 128)
        self.encoder3 = self.conv_block(128, 256)
        self.encoder4 = self.conv_block(256, 512)

        # Bottleneck
        self.bottleneck = self.conv_block(512, 1024)

        # Decoder (Upsampling path)
        self.upconv4 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.decoder4 = self.conv_block(1024, 512)
        self.upconv3 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.decoder3 = self.conv_block(512, 256)
        self.upconv2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.decoder2 = self.conv_block(256, 128)
        self.upconv1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.decoder1 = self.conv_block(128, 64)

        # Final output layer
        self.final_layer = nn.Conv2d(64, 1, kernel_size=1)

    def conv_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        # Encoder path
        enc1 = self.encoder1(x)
        enc2 = self.encoder2(self.maxpool(enc1))
        enc3 = self.encoder3(self.maxpool(enc2))
        enc4 = self.encoder4(self.maxpool(enc3))

        # Bottleneck
        bottleneck = self.bottleneck(self.maxpool(enc4))

        # Decoder path
        dec4 = self.upconv4(bottleneck)
        dec4 = torch.cat((enc4, dec4), dim=1)
        dec4 = self.decoder4(dec4)
        dec3 = self.upconv3(dec4)
        dec3 = torch.cat((enc3, dec3), dim=1)
        dec3 = self.decoder3(dec3)
        dec2 = self.upconv2(dec3)
        dec2 = torch.cat((enc2, dec2), dim=1)
        dec2 = self.decoder2(dec2)
        dec1 = self.upconv1(dec2)
        dec1 = torch.cat((enc1, dec1), dim=1)
        dec1 = self.decoder1(dec1)

        # Final layer
        return self.final_layer(dec1)

    @property
    def maxpool(self):
        return nn.MaxPool2d(kernel_size=2, stride=2)

# Create the U-Net model
model = UNet()
print(model)

# Example input
x = torch.randn(1, 1, 1216, 192)
output = model(x)
print(output.shape)


UNet(
  (encoder1): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
  )
  (encoder2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
  )
  (encoder3): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=T

In [ ]:
import torch
import torch.nn as nn

class SymmetricCNN(nn.Module):
    def __init__(self):
        super(SymmetricCNN, self).__init__()
        
        # Define the convolutional layers
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)  # Output: (32, 192, 1216)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)  # Output: (64, 192, 1216)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)  # Output: (128, 192, 1216)
        
        # Define the transposed convolutional layers (to upscale back to original size)
        # self.deconv1 = nn.ConvTranspose2d(128, 64, kernel_size=3, padding=1)  # Output: (64, 192, 1216)
        # self.deconv2 = nn.ConvTranspose2d(64, 32, kernel_size=3, padding=1)  # Output: (32, 192, 1216)
        # self.deconv3 = nn.ConvTranspose2d(32, 1, kernel_size=3, padding=1)  # Output: (1, 192, 1216)
        
        # Activation function
        self.relu = nn.ReLU()
        
    def forward(self, x):
        # Apply convolutions
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.relu(self.conv3(x))
        
        # Apply transposed convolutions
        x = self.relu(self.deconv1(x))
        x = self.relu(self.deconv2(x))
        x = self.deconv3(x)  # No ReLU on the last layer to maintain original value range
        
        return x

class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.linear1 = nn.Linear(192, 192*4)
        self.linear2 = nn.Linear(192*4, 192)
        # Activation function
        self.relu = nn.ReLU()
        
    def forward(self, x):
        # Apply convolutions
        x = self.relu(self.linear1(x))
        x = self.linear2(x)
        
        return x
# Example usage:
model = SymmetricCNN()
input_tensor = torch.randn(1, 1, 192, 1216)  # Batch size 1, 1 channel, 192x1216 image
output_tensor = model(input_tensor)
print(output_tensor.shape)  # Should be torch.Size([1, 1, 192, 1216])


In [5]:
from PIL import Image
import os

output_dir = 'output_images'

def img_process(image):
    # image = image.transpose(1, 0)
    # Convert numpy arrays to PIL Images
    image = (image).astype(np.uint8)
    pil_image = Image.fromarray(image)
    return pil_image

def save_img(batch_data, outputs, batch_labels=None, num=0, output_dir='output_images'):
    os.makedirs(output_dir, exist_ok=True)
    for j in range(batch_data.size(0)):
        input_image = batch_data[j].detach().squeeze().cpu().numpy()
        output_image = outputs[j].detach().squeeze().cpu().numpy()
        
        input_pil = img_process(input_image)
        output_pil = img_process(output_image)
        
        # Save images
        input_pil.save(os.path.join(output_dir, f'input_{num * batch_data.size(0) + j}.png'))
        output_pil.save(os.path.join(output_dir, f'output_{num * batch_data.size(0) + j}.png'))

        if batch_labels!=None:
            label_image = batch_labels[j].squeeze().cpu().numpy()
            label_pil = img_process(label_image)
            label_pil.save(os.path.join(output_dir, f'label_{num * batch_data.size(0) + j}.png'))

In [6]:
import time

num_epochs = 1
time_start = time.time()
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
model.to(device)#.half()
for epoch in range(num_epochs):
    # train
    running_loss = 0
    model.train()
    for i, (batch_data, batch_labels) in enumerate(train_loader):
        # print(batch_data.shape)
        # print(batch_labels.shape)
        batch_data = batch_data.to(device)#.float()
        batch_labels = batch_labels.to(device)#.float()

        out = model(batch_data)#.unsqueeze(1)
        # out = model(batch_data).permute(0, 2, 1).unsqueeze(1)
        # print(out.shape)
        # print(batch_labels.shape)
        loss = criterion(out, batch_labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        if i % 100 == 0:
            print("epoch: {}/{}, batch: {}, Loss: {}, Time:{}".format(epoch+1, num_epochs, i, loss.data, time.time()-time_start))
            save_img(batch_data, out, batch_labels=batch_labels, num=i, output_dir='output_images_train')    
            time_start = time.time()
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}')

    # val
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for i, (batch_data, batch_labels) in enumerate(val_loader):
            batch_data = batch_data.to(device)
            batch_labels = batch_labels.to(device)
            out = model(batch_data)#.unsqueeze(1)
            # out = model(batch_data).permute(0, 2, 1).unsqueeze(1)
            loss = criterion(out, batch_labels)
            val_loss += loss.item()
        
    print(f'Val Loss: {val_loss/len(val_loader)}')


epoch: 1/1, batch: 0, Loss: 178.57675170898438, Time:5.486355543136597
epoch: 1/1, batch: 100, Loss: 155.3648681640625, Time:22.626109838485718
epoch: 1/1, batch: 200, Loss: 151.6075897216797, Time:22.69310975074768
epoch: 1/1, batch: 300, Loss: 156.5375518798828, Time:22.762123107910156
epoch: 1/1, batch: 400, Loss: 133.0918731689453, Time:22.80213212966919
epoch: 1/1, batch: 500, Loss: 104.75497436523438, Time:22.841140270233154
epoch: 1/1, batch: 600, Loss: 85.1009750366211, Time:22.85864806175232
epoch: 1/1, batch: 700, Loss: 79.78836059570312, Time:22.862650394439697
epoch: 1/1, batch: 800, Loss: 70.32545471191406, Time:22.805132627487183
Epoch 1/1, Loss: 118.13265032087054
Val Loss: 71.27994116210938


In [8]:
torch.save(model, "./UNet_50.pth")

In [ ]:
from PIL import Image
import os

output_dir = 'output_images'
os.makedirs(output_dir, exist_ok=True)

# val
model.eval()
test_loss = 0
with torch.no_grad():
    for i, (batch_data, batch_labels) in enumerate(train_data_loader):
        for j in range(batch_data.size(0)):
            input_image = batch_data[j].squeeze().cpu().numpy()
            output_image = batch_labels[j].squeeze().cpu().numpy()

            input_image = input_image.transpose(1, 0)
            output_image = output_image.transpose(1, 0)
            
            # Convert numpy arrays to PIL Images
            input_image = (input_image).astype(np.uint8)
            output_image = (output_image).astype(np.uint8)
            
            input_pil = Image.fromarray(input_image)
            output_pil = Image.fromarray(output_image)
            
            # Save images
            input_pil.save(os.path.join(output_dir, f'input_{i * batch_data.size(0) + j}.png'))
            output_pil.save(os.path.join(output_dir, f'output_{i * batch_data.size(0) + j}.png'))
        if(i==10): break
print(f'Test Loss: {test_loss/len(test_loader)}')

In [ ]:
model.printInfo()

In [9]:
from PIL import Image
import os

output_dir = 'output_images_test'
os.makedirs(output_dir, exist_ok=True)
outList = np.zeros([seq_test, 1216, 192])
# test
model.eval()
test_loss = 0
with torch.no_grad():
    for i, (batch_data, batch_labels) in enumerate(test_loader):
        batch_data = batch_data.to(device)
        batch_labels = batch_labels.to(device)
        out = model(batch_data)
        outList[i*batch_size:(i+1)*batch_size] = out.detach().squeeze().cpu().numpy()
        loss = criterion(out.unsqueeze(1), batch_labels)
        test_loss += loss.item()
        save_img(batch_data, out, batch_labels=batch_labels, num=i, output_dir=output_dir)    
    
print(f'Test Loss: {test_loss/len(test_loader)}')

Test Loss: 71.22007745361329


In [ ]:
# val
model.eval()
test_loss = 0
with torch.no_grad():
    for i, (batch_data, batch_labels) in enumerate(test_loader):
        out = model(batch_data)
        loss = criterion(out, batch_labels)
        test_loss += loss.item()
    
print(f'Test Loss: {test_loss/len(test_loader)}')